In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AutoConfig, BertForSequenceClassification

from sklearn.neighbors import NearestNeighbors

In [ ]:
ON_STAGE2 = True  # setup False to close Stage2
N_NEIGHBOR = 50 # count of Stage1 retrival

# INPUT_DIR = '/kaggle/input/learning-equality-curriculum-recommendations'
OUTPUT_PATH = '/home/search3/lichunyu/k12-curriculum-recommendations/tmp/output'

# TOPIC_DIR = os.path.join(INPUT_DIR, 'topics.csv')
# CONTENT_DIR = os.path.join(INPUT_DIR, 'content.csv')
# CORR_DIR = os.path.join(INPUT_DIR, 'sample_submission.csv')

MODEL_DIR = '/home/search3/lichunyu/k12-curriculum-recommendations/data/input/models/stage1/all-MiniLM-L6-v2/r1_12_38520/checkpoint-38520'
TOKENIZER_DIR = '/home/search3/lichunyu/k12-curriculum-recommendations/data/input/models/stage1/all-MiniLM-L6-v2/all-MiniLM-L6-v2/all-MiniLM-L6-v2_new'

In [ ]:
def get_topic_field(d):
    title = list(filter(lambda x: pd.notna(x), d['title_level']))
    title = ' of '.join(title[-1::-1])
    title = 'No information' if title=='' else title
    title = '[TITLE] ' + title + '. '
    description = d['description'] if pd.notna(d['description']) else 'No information'
    description = '[DESCRIPTION]' + description + '. '
    field = title + description
    return field

def get_content_field(d):
    title = d['title']
    title = 'No information' if pd.isna(title) else title
    title = '[TITLE] ' + title + '. '
    description = d['description'] if pd.notna(d['description']) else 'No information'
    description = '[DESCRIPTION]' + description + '. '
    kind = '[' + d['kind'] + '] '
    field = kind + title + description
    return field

In [ ]:
class DataPreparation:
    
    def __init__(self, topic_path, content_path, submission_path):
        self.topic = pd.read_csv(topic_path)
        self.content = pd.read_parquet(content_path)
        self.corr = pd.read_parquet(submission_path).drop_duplicates(subset="topic_id").reset_index(drop=True)
        # self.topic = self.topic[self.topic['id'].isin(self.corr['topic_id'].to_list())]
        self.match_dict = None
    
    def prepare_topic(self):
        df_level = self._get_level_features(self.topic)
        self.topic = self.topic.merge(df_level, on='id', how='inner')
        self.topic['field'] = self.topic.apply(lambda x: get_topic_field(x), axis=1)
        return self.topic
    
    def prepare_content(self):
        self.content['field'] = self.content.apply(lambda x: get_content_field(x), axis=1)
        return self.content
    
    def prepare_language_match(self):
        topic = self.topic[['id', 'language']].merge(self.corr, left_on='id', right_on='topic_id', how='right')[['id', 'language']]
        match_dict = {}
        print(topic['language'].unique())
        for language in topic['language'].unique():
            match_dict[language] = (topic.query('language==@language')[['id']], self.content.query('language==@language')[['id']])
        self.match_dict = match_dict
        return match_dict
    
    
    def _get_level_features(self, df_topic, level_cols=['title']):
        cols = list(set(level_cols + ['id', 'parent', 'level', 'has_content']))
        df_hier = df_topic[cols]
        
        highest_level = df_hier['level'].max()
        print(f'Highest Level: {highest_level}')

        df_level = df_hier.query('level == 0').copy(deep=True)
        level_list = list()
        for col in level_cols:
            df_level[f'{col}_level'] = df_level[f'{col}'].apply(lambda x: [x])

        for i in tqdm(range(highest_level + 1)):
            level_list.append(df_level[df_level['has_content']])
            df_level_high = df_hier.query('level == @i+1')
            df_level = df_level_high.merge(df_level, left_on='parent', right_on='id', suffixes=['', '_parent'], how='inner')
            for col in level_cols:
                df_level[f'{col}_level'] = df_level[f'{col}_level'] + df_level[f'{col}'].apply(lambda x: [x])
            for col in df_level.columns:
                if col.endswith('_parent'):
                    df_level.drop(columns=col, inplace=True)
        df = pd.concat(level_list).reset_index(drop=True)
        return df[set(['id'] + [f'{col}_level' for col in level_cols])]
    
    def prepare(self):
        self.prepare_topic()
        self.prepare_content()
        self.prepare_language_match()

In [ ]:
%%time

# Train
# topic_dir = "/home/search3/lichunyu/k12-curriculum-recommendations/data/input/raw/topics.csv"
# content_dir = "/home/search3/lichunyu/k12-curriculum-recommendations/data/input/kflod_data/flod0/train_content_flod0.pqt"
# corr_dir = "/home/search3/lichunyu/k12-curriculum-recommendations/data/input/kflod_data/flod0/train_correlations_flod0.pqt"

# Valid
topic_dir = "/home/search3/lichunyu/k12-curriculum-recommendations/data/input/raw/topics.csv"
content_dir = "/home/search3/lichunyu/k12-curriculum-recommendations/data/input/kflod_data/flod0/valid_content_flod0.pqt"
corr_dir = "/home/search3/lichunyu/k12-curriculum-recommendations/data/input/kflod_data/flod0/valid_correlations_flod0_no_source.pqt"


dp = DataPreparation(topic_dir, content_dir, corr_dir)
x = dp.prepare()

# Retrival

In [ ]:
class PlainDataset(Dataset):

    def __init__(self, df, tokenizer, label_name="") -> None:
        super().__init__()
        self.data = df[label_name].tolist()
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        text = self.data[index]
        inputs = self.tokenizer(
                text, 
                add_special_tokens = True,
                truncation='longest_first',
                max_length = 128,
                padding = 'max_length',
                return_attention_mask = True,
                return_tensors = 'pt',
        )
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs

    def __len__(self):
        return len(self.data)

In [ ]:
class Retrieval():
    
    def __init__(self, model_path, tokenizer_path, dp):
        self.model = AutoModel.from_pretrained(model_path).cuda()
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
        
        self.topic = dp.topic
        self.content = dp.content
        self.corr = dp.corr
        self.topic_content_match = dp.match_dict
    
    def convert2embed(self, df, label_name='field'):
        embed: list = []
        dataset = PlainDataset(df, tokenizer=self.tokenizer, label_name=label_name)
        dataloader = DataLoader(dataset, sampler=SequentialSampler(dataset), batch_size=32)
        with torch.no_grad():
            for batch in dataloader:
                batch = {k: v.cuda() for k, v in batch.items()}
                embeddings = self.model(**batch, output_hidden_states=True, return_dict=True).pooler_output
                embed.append(embeddings.cpu().clone().detach().numpy())
        embed = np.concatenate(embed, axis=0)
        return embed
    
    def get_embed(self):
        for lang in self.topic_content_match.keys():
            if not isinstance(lang, str):
                continue
            topic_, content_ = self.topic_content_match[lang]
            topic_ = topic_[['id']].merge(self.topic[['id', 'field']], on='id', how='left')
            content_ = content_[['id']].merge(self.content[['id', 'field']], on='id', how='left')
            
            topic_path = os.path.join(OUTPUT_PATH, f"topic_{lang}.pqt")
            content_path = os.path.join(OUTPUT_PATH, f"content_{lang}.pqt")
            topic_.to_parquet(topic_path)
            content_.to_parquet(content_path)
            
            for t in ["content", "topic"]:
                path = os.path.join(OUTPUT_PATH, f"{t}_{lang}.pqt")
                df = pd.read_parquet(path)
                embed = self.convert2embed(df, label_name="field")
                np.save(path.replace(".pqt", ".npy"), embed)
                
    def inference(self):
        recall_amount = 0
        recall_amount_total = 0
        recall_num = 0
        recall_total = {}
        
        df_pred_list = []
        for lang in self.topic_content_match.keys():
            if not isinstance(lang, str):
                continue
            # global df_pred, df_correlations
            content_path = os.path.join(OUTPUT_PATH, f"content_{lang}.npy")
            topics_path = os.path.join(OUTPUT_PATH, f"topic_{lang}.npy")
            content_array = np.load(content_path)
            topics_array = np.load(topics_path)
            
            model = NearestNeighbors(n_neighbors=N_NEIGHBOR, metric="cosine")
            model.fit(content_array)
            d, r = model.kneighbors(topics_array)
            df_content = pd.read_parquet(content_path.replace(".npy", ".pqt"))
            df_topics = pd.read_parquet(topics_path.replace(".npy", ".pqt"))
            df_correlations = self.corr

            pred = {"topic_id": [], "content_ids": []}
            for i in range(len(df_topics)):
                r_t = r[i]
                tmp = []
                for c in r_t:
                    content_id = df_content.iloc[c]["id"]
                    tmp.append(content_id)
                topics_id = df_topics.iloc[i]["id"]
                pred["topic_id"].append(topics_id)
                pred["content_ids"].append(tmp)

            df_pred = pd.DataFrame(pred).astype({"topic_id": str})
            df_pred_list.append(df_pred)
        df_pred = pd.concat(df_pred_list)
        self.df_pred = df_pred
        self.df_pred['content_ids'] = self.df_pred.apply(lambda x: ' '.join(x['content_ids']), axis=1)
        
    def save_pred(self, path='submission.csv'):
        self.df_pred.to_csv(path, index=None)
                
    def retrieval(self):
        self.get_embed()
        self.inference()
        self.save_pred()
        return self.df_pred

In [ ]:
%%time
s1 = Retrieval(MODEL_DIR, TOKENIZER_DIR, dp)
df_retrival = s1.retrieval()

In [ ]:
df_retrival = df_retrival.reset_index(drop=True)

In [ ]:
df_retrival

In [ ]:
df_retrival.to_parquet("/home/search3/lichunyu/k12-curriculum-recommendations/data/output/stage2/valid/raw_submission.pqt")

In [ ]:
dp.topic.to_parquet("/home/search3/lichunyu/k12-curriculum-recommendations/data/output/stage2/valid/topic.pqt")

In [ ]:
dp.content.to_parquet("/home/search3/lichunyu/k12-curriculum-recommendations/data/output/stage2/valid/content.pqt")

In [ ]:
df_retrival["content_ids"] = df_retrival["content_ids"].apply(lambda x: x.split())
df_retrival = df_retrival.explode("content_ids").reset_index(drop=True)

df_retrival = df_retrival.merge(dp.topic[["id", "field"]], left_on="topic_id", right_on="id", how="left")[
    ["topic_id", "content_ids", "field"]
].rename({"field": "topic_field"}, axis=1)
df_retrival = df_retrival.merge(dp.content[["id", "field"]], left_on="content_ids", right_on="id", how="left")[
    ["topic_id", "content_ids", "topic_field", "field"]
].rename({"field": "content_field"}, axis=1)

df_retrival

In [ ]:
label = pd.read_parquet(corr_dir)
label["label"] = 1
label

In [ ]:
df_retrival = df_retrival.merge(label, on=["topic_id", "content_ids"], how="left").fillna({"label": 0})
df_retrival["label"] = df_retrival["label"].astype("int")
df_retrival

In [ ]:
df_retrival.to_parquet("/home/search3/lichunyu/k12-curriculum-recommendations/data/output/stage2/valid/valid.pqt")

In [1]:
import torch

/home/search3/miniconda3/envs/kaggle/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
torch.Tensor([1])

tensor([1.])

In [6]:
import pandas as pd
pd.DataFrame({"a": [1,2,3]}).sort_values("a", ascending=False)

,a
2,3
1,2
0,1
